# Teste para predizer as labels das avaliações com 3 e 4.

Realizei alguns testes para predizer as labels do dataset com outros algoritmos sem ser o BERT, mas a acurácia não ficou em um valor aceitável.

In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
df_reviews = pd.read_csv("drive/My Drive/Colab Notebooks/TCC/default_dataset.csv")
df_reviews_test = pd.read_csv("drive/My Drive/Colab Notebooks/TCC/three_and_four_labeled.csv")

In [ ]:
df_reviews.head()

,reviewer,rating,written_date,title,review_text,branch,month,year,day_of_week,label
0,Kelly B,2.0,2021-05-30,Universal is a complete Disaster - stick with ...,We went to Universal over Memorial Day weekend...,Universal Studios Florida,5,2021,6,0.0
1,Jon,1.0,2021-05-30,Food is hard to get.,The food service is horrible. I’m not reviewin...,Universal Studios Florida,5,2021,6,0.0
2,Nerdy P,2.0,2021-05-30,Disappointed,I booked this vacation mainly to ride Hagrid m...,Universal Studios Florida,5,2021,6,0.0
3,tammies20132015,5.0,2021-05-28,The Bourne Stuntacular...MUST SEE,"Ok, I can't stress enough to anyone and everyo...",Universal Studios Florida,5,2021,4,1.0
4,John,1.0,2021-05-28,This is not a vacation,Worst experience I have ever had the rides are...,Universal Studios Florida,5,2021,4,0.0


In [ ]:
classlabel_count = df_reviews.label.value_counts()
print('Class 0:', classlabel_count[0])
print('Class 1:', classlabel_count[1])
print('Proportion:', round(classlabel_count[0] / classlabel_count[1], 2), ': 1')

Class 0: 3959
Class 1: 28202
Proportion: 0.14 : 1


In [ ]:
# Load language and make functions to help to prepare
nlp_lemma = spacy.load('en_core_web_sm')

# Convert to array
def convert_text_to_array(text):
    text_converted = nlp_lemma(text)
    
    return [each_word for each_word in text_converted]

# Get token and lemmatization
def get_token_and_lemma(text):
    return [each_word.lemma_ for each_word in text]

# remove stop words and not users citation
def remove_stop_words(tokens):
    return [each_word for each_word in tokens \
            if not nlp_lemma.vocab[each_word].is_stop \
            and nlp_lemma.vocab[each_word].is_alpha \
            and each_word[:1] != "@"]

In [ ]:
# get tokens in the text and remove stop words

labels = df_reviews["label"].unique()
samples_balanced_reviews = pd.DataFrame()

for each_label in labels:
    df_this_rate = df_reviews[df_reviews["label"] == each_label].sample(n=2500)
    samples_balanced_reviews = samples_balanced_reviews.append(df_this_rate)

In [ ]:
samples_balanced_reviews.head()

,reviewer,rating,written_date,title,review_text,branch,month,year,day_of_week,label
18418,dwinn717,1.0,2013-10-01,Careful if you are Plus Sized!!,"I am making this review ""terrible"" because I a...",Universal Studios Florida,10,2013,1,0.0
22122,Ikebukero,2.0,2016-07-31,Queueing 3 h for Harry,"went on a monday, Queueing 3 h for Harry, to m...",Universal Studios Japan,7,2016,6,0.0
11194,santo19,1.0,2016-03-19,Even worse as time goes by,The park was a messy. Garbage for all place. S...,Universal Studios Florida,3,2016,5,0.0
3771,Hans-Hungrig,1.0,2018-07-09,Fast and furious,Awful. Express line does not exist. Queued for...,Universal Studios Florida,7,2018,0,0.0
20248,meliee_sorciere,2.0,2010-10-15,"Old, boring, overpriced","I visited this park on September 2010, the par...",Universal Studios Florida,10,2010,4,0.0


In [ ]:
samples_balanced_reviews['tokens'] = samples_balanced_reviews['review_text'].apply(lambda x: convert_text_to_array(x))
samples_balanced_reviews['tokens'] = samples_balanced_reviews['tokens'].apply(lambda x: get_token_and_lemma(x))
samples_balanced_reviews['tokens'] = samples_balanced_reviews['tokens'].apply(lambda x: remove_stop_words(x))

In [ ]:
def dummy(doc):
    return doc

vectorizer = TfidfVectorizer(tokenizer=dummy, preprocessor=dummy, ngram_range=(1,2))

train_vectors = vectorizer.fit_transform(samples_balanced_reviews["tokens"])

In [ ]:
df_reviews_test['tokens'] = df_reviews_test['review_text'].apply(lambda x: convert_text_to_array(x))
df_reviews_test['tokens'] = df_reviews_test['tokens'].apply(lambda x: get_token_and_lemma(x))
df_reviews_test['tokens'] = df_reviews_test['tokens'].apply(lambda x: remove_stop_words(x))

test_vectors = vectorizer.transform(df_reviews_test["tokens"])

In [ ]:
clf = LogisticRegression(C=10000.0, class_weight='balanced', solver='newton-cg', 
                         n_jobs=-1, max_iter=1000)

scores = model_selection.cross_val_score(clf, train_vectors, samples_balanced_reviews["label"], cv=7, scoring="f1")

print({"mean": scores.mean(), "std": scores.std(), "max" : scores.max()})
    
clf.fit(train_vectors, samples_balanced_reviews["label"])
predicted_labels = clf.predict(test_vectors)

{'mean': 0.923819866065811, 'std': 0.004474200976052992, 'max': 0.9294605809128631}


A acuracia do modelo se manteve abaixo de 75% para todas as vezes que testei, por esta questão, decidi procurar por um modelo melhor, que se adaptaria mais à NLP, e também à língua inglesa. Após uma pesquisa na internet, cheguei ao BERT.

In [ ]:
acuracia = np.count_nonzero(np.array(df_reviews_test["label"] == predicted_labels))/np.array(df_reviews_test["label"] == predicted_labels).shape[0]
print(acuracia)

0.73125


In [ ]:
df_review_without_label = pd.read_csv("drive/My Drive/Colab Notebooks/TCC/dataset_without_labels.csv")

In [ ]:
df_review_without_label['tokens'] = df_review_without_label['review_text'].apply(lambda x: convert_text_to_array(x))
df_review_without_label['tokens'] = df_review_without_label['tokens'].apply(lambda x: get_token_and_lemma(x))
df_review_without_label['tokens'] = df_review_without_label['tokens'].apply(lambda x: remove_stop_words(x))

In [ ]:
vectors_to_predict = vectorizer.transform(df_review_without_label["tokens"])

In [ ]:
predicted_labels = clf.predict(vectors_to_predict)

In [ ]:
df_review_without_label['label'] = predicted_labels

In [ ]:
df_review_without_label.to_csv("drive/My Drive/Colab Notebooks/TCC/analysis/dataset_without_labels_labeled_logistic.csv")